# OHBM 2024 educational session -  Molecular Imaging: a Hands-on Tutorial based on Open Access Datasets

In the following Jupyter notebook, we will introduce you the basics of PET preprocessing using BIDS compliant data. You will download some PET data from OpenNeuro and then work on preprocessing.

## Running processing pipelines on BIDS data
Note, run this only only if freesurfer is installed (you need to run the cells below)

In [1]:
import subprocess
check_for_tree = subprocess.run(['which', 'tree'], capture_output=True)
if check_for_tree.returncode == 0:
    pass
else:
    import platform
    operating_system = platform.system()
    if operating_system == 'Linux':
        subprocess.run("apt-get install tree -y", shell=True)
    elif operating_system == 'Darwin':
        subprocess.run("brew install tree", shell=True)
    else:
        print("You're on your own windows user.")

## Install FreeSurfer
Note, this is only necessary if you want to actually run the pipelines below


In [32]:
# Download freesurfer 7.4.1
!wget https://surfer.nmr.mgh.harvard.edu/pub/dist/freesurfer/7.4.1/freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz
!if [ -f freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz ]; then time pigz -dc freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz | tar xf -; fi
import os, sys
# append freesurfer to system path
freesurfer_path = os.path.join(os.getcwd(), 'freesurfer/bin/')
os.getenv('PATH')
if freesurfer_path not in os.environ['PATH']:
  os.environ['PATH'] = freesurfer_path + ':' + os.getenv('PATH')
freesurfer_home = os.environ['FREESURFER_HOME'] = os.path.join(os.getcwd(), 'freesurfer')
subjects_dir = os.environ['SUBJECTS_DIR'] = os.path.join(os.getcwd(), 'freesurfer', 'subjects')

def set_global_variables():
    global freesurfer_home
    global subjects_dir

set_global_variables()

# copy license file into freesurfer home
!cp license.txt $FREESURFER_HOME/
!echo $PATH
!mri_coreg --version


--2024-06-12 16:06:48--  https://surfer.nmr.mgh.harvard.edu/pub/dist/freesurfer/7.4.1/freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz
Løser surfer.nmr.mgh.harvard.edu (surfer.nmr.mgh.harvard.edu) … 132.183.1.43
Tilslutter surfer.nmr.mgh.harvard.edu (surfer.nmr.mgh.harvard.edu)|132.183.1.43|:443 … forbundet.
HTTP forespørgsel sendt, afventer svar … 200 OK
Længde: 9484461482 (8.8G) [application/x-gzip]
Gemmer til: 'freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz'

surfer-linux-ubuntu  12%[=>                  ]   1.10G  3.53MB/s    tid 39m 36s^C
/bin/bash: pigz: command not found

real	0m0.012s
user	0m0.001s
sys	0m0.003s
cp: directory /Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/outreach/OHBM2024/freesurfer does not exist
/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/outreach/OHBM2024/freesurfer/bin/:/Users/martinnorgaard/anaconda3/envs/OHBM2024/bin:/Users/martinnorgaard/anaconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/opt/fsl/bin:/usr/local/bin:/Applications/freesurfer/7.4.1

In [5]:
print(freesurfer_home)

/content/outreach/StratNeuro2024/freesurfer


In [ ]:
!mkdir PETprocessing

In [ ]:
!cd PETprocessing

In [ ]:
!if [ -d "PET_pipelines" ]; then pushd PET_pipelines && git pull; else git clone https://github.com/openneuropet/PET_pipelines.git; fi

In [ ]:
!ls PET_pipelines/pyPetSurfer

In [ ]:
%cd PET_pipelines/pyPetSurfer
!wget /dev/null https://www.dropbox.com/sh/69dwtnv29wd7jlx/AADnw5FvAANpvzKAxVQTnyhBa?dl=0
!if [ -f AADnw5FvAANpvzKAxVQTnyhBa?dl=0 ]; then unzip -o AADnw5FvAANpvzKAxVQTnyhBa?dl=0; fi
!if [ ! -f AADnw5FvAANpvzKAxVQTnyhBa?dl=0 ]; then chmod +x ds001421-1.4.1.sh && mkdir ds001421-download/; fi
!if [ ! -f AADnw5FvAANpvzKAxVQTnyhBa?dl=0 ]; then cp ds001421-1.4.1.sh ds001421-download/; fi
!if [ ! -f AADnw5FvAANpvzKAxVQTnyhBa?dl=0 ]; then cd ds001421-download/ && ./ds001421-1.4.1.sh && rm ds001421-1.4.1.sh; fi

In [ ]:
os.environ['FREESURFER_HOME'] = freesurfer_home
os.environ['SUBJECTS_DIR'] = subjects_dir
!ls && python3 example.py

In [ ]:
!ls .

In [4]:
import subprocess
import os

# Define a function to run shell commands and handle errors
def run_command(command, input=None, cwd=None):
    try:
        result = subprocess.run(command, input=input, text=True, capture_output=True, shell=True, cwd=cwd)
        print(result.stdout)
        if result.stderr:
            print("Error:", result.stderr)
    except Exception as e:
        print("An error occurred:", e)

# Download FSL installer script
print("Downloading FSL installer...")
run_command("wget -O fslinstaller.py https://fsl.fmrib.ox.ac.uk/fsldownloads/fslinstaller.py")

# Run the FSL installer with non-interactive flags
print("Running FSL installer non-interactively...")
installer_command = "python fslinstaller.py --dest=/opt/fsl --skip_registration"
run_command(installer_command)

# Set environment variables for the session
fsl_dir = "/opt/fsl"  # Adjust this path according to your installation
os.environ['FSLDIR'] = fsl_dir
os.environ['PATH'] += f":{fsl_dir}/bin"
os.environ['FSLOUTPUTTYPE'] = "NIFTI_GZ"

# Source FSL configuration script (this step might not fully work in Jupyter as mentioned before)
print("Configuring FSL environment...")
run_command(f"source {fsl_dir}/etc/fslconf/fsl.sh")

print("FSL Installation and configuration complete.")


Error: --2024-06-12 14:16:09--  https://fsl.fmrib.ox.ac.uk/fsldownloads/fslinstaller.py
Løser fsl.fmrib.ox.ac.uk (fsl.fmrib.ox.ac.uk) … 129.67.248.65
Tilslutter fsl.fmrib.ox.ac.uk (fsl.fmrib.ox.ac.uk)|129.67.248.65|:443 … forbundet.
HTTP forespørgsel sendt, afventer svar … 200 OK
Længde: 74517 (73K) [application/octet-stream]
Gemmer til: 'fslinstaller.py'

     0K .......... .......... .......... .......... .......... 68%  106K 0s
    50K .......... .......... ..                              100%  239M=0.5s

2024-06-12 14:16:10 (154 KB/s) — 'fslinstaller.py' gemt [74517/74517]


Running FSL installer non-interactively...
FSL installer version: 3.2.0
Press CTRL+C at any time to cancel installation

Error: usage: fslinstaller.py [-h] [-v] [-d DESTDIR] [-o] [-l] [-n] [-s] [-m]
                       [-V FSLVERSION]
fslinstaller.py: error: unrecognized arguments: --skip_registration

Configuring FSL environment...

FSL Installation and configuration complete.


FLIRT version 6.0

Usage: flirt [options] -in <inputvol> -ref <refvol> -out <outputvol>
       flirt [options] -in <inputvol> -ref <refvol> -omat <outputmatrix>
       flirt [options] -in <inputvol> -ref <refvol> -applyxfm -init <matrix> -out <outputvol>

  Available options are:
        -in  <inputvol>                    (no default)
        -ref <refvol>                      (no default)
        -init <matrix-filname>             (input 4x4 affine matrix)
        -omat <matrix-filename>            (output in 4x4 ascii format)
        -out, -o <outputvol>               (default is none)
        -datatype {char,short,int,float,double}                    (force output data type)
        -cost {mutualinfo,corratio,normcorr,normmi,leastsq,labeldiff,bbr}        (default is corratio)
        -searchcost {mutualinfo,corratio,normcorr,normmi,leastsq,labeldiff,bbr}  (default is corratio)
        -usesqform                         (initialise using appropriate sform or qform)
        -displayin

In [8]:
import os
import subprocess

def run_command(command, cwd=None):
    """ Helper function to run a command in the specified working directory. """
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True, cwd=cwd)
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print("Error:", result.stderr)

# Clone the repository
repo_url = "https://github.com/mnoergaard/petprep_hmc.git"
clone_directory = "."  # Change this to your desired path
clone_command = f"git clone {repo_url}"
print("Cloning repository...")
run_command(clone_command, cwd=clone_directory)

# Change to the repository directory
repo_path = os.path.join(clone_directory, "petprep_hmc")
os.chdir(repo_path)
print(f"Changed directory to {repo_path}")

# Assuming the repository has a requirements.txt file or similar setup script
# Install Python dependencies
print("Installing dependencies...")
run_command("pip install -r ./petprep_hmc/environment.yml")

print("Repository setup complete.")

Cloning repository...
Error: Cloning into 'petprep_hmc'...
Updating files:  64% (62/96)
Updating files:  65% (63/96)
Updating files:  66% (64/96)
Updating files:  67% (65/96)
Updating files:  68% (66/96)
Updating files:  69% (67/96)
Updating files:  70% (68/96)
Updating files:  71% (69/96)
Updating files:  72% (70/96)
Updating files:  73% (71/96)
Updating files:  75% (72/96)
Updating files:  76% (73/96)
Updating files:  77% (74/96)
Updating files:  78% (75/96)
Updating files:  79% (76/96)
Updating files:  80% (77/96)
Updating files:  81% (78/96)
Updating files:  82% (79/96)
Updating files:  83% (80/96)
Updating files:  84% (81/96)
Updating files:  85% (82/96)
Updating files:  86% (83/96)
Updating files:  87% (84/96)
Updating files:  88% (85/96)
Updating files:  89% (86/96)
Updating files:  90% (87/96)
Updating files:  91% (88/96)
Updating files:  92% (89/96)
Updating files:  93% (90/96)
Updating files:  94% (91/96)
Updating files:  95% (92/96)
Updating files:  96% (93/96)
Updating file

In [18]:
# Assuming the repository has a requirements.txt file or similar setup script
# Install Python dependencies
print("Installing dependencies...")
run_command("pip install -e .")

print("Repository setup complete.")

Installing dependencies...
Obtaining file:///Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/outreach/OHBM2024/petprep_hmc
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached click-8.1.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached networkx-3.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached nibabel-5.1.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached patsy-0.5.3-py2.py3-none-any.whl.metadata (3.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/53.9 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.6 MB/s eta 0:00:00
  Using cached seaborn-0.12.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached bids-0.0-py2.py3-none-any.whl.metadata (352 bytes)
  Using cached bids_validator-1.11.0-py2.py3-none-any.whl.metadata (38 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
  Using cached formulaic-0.5.2-py3-none-any.w

In [41]:
!python3 run.py --bids_dir /Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/outreach/OHBM2024/data --output_dir /Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/outreach/OHBM2024/data/derivatives/petprep_hmc --n_procs 4

Traceback (most recent call last):
  File "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/outreach/OHBM2024/code/petprep_hmc/run.py", line 601, in <module>
    main(args)
  File "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/outreach/OHBM2024/code/petprep_hmc/run.py", line 40, in main
    raise Exception('You need a valid FreeSurfer license to proceed!')
Exception: You need a valid FreeSurfer license to proceed!


In [40]:
ls

Dockerfile            dist/                 run.py
LICENSE               docs/                 scripts/
README.md             environment.yml       setup.py
build/                petprep_hmc/          tests/
data/                 petprep_hmc.egg-info/ version
